Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 </h2>
    <h1> #Constitución </h1>
    <p>
        Profesor Vicente Domínguez<br>
        Primer semestre 2022<br>    
        Fecha de entrega: Viernes 1 de Julio a las 22:00hrs.
    </p>
    <br>
</center>

<br>

---

## Disclaimer
La intención de esta tarea no es polarizar ni propagar una agenda política. Su objetivo es analizar de manera imparcial los datos que se tienen. Creemos que este es un tema de interés y contingente para todo estudiante, y que es de mucha utilidad mostrarles cómo son aplicables las herramientas enseñadas en el curso en un caso real. Recuerda siempre mantener la cordialidad y objetividad en el momento del análisis.

## Disclaimer 2
Al leer esto estás de acuerdo con no compartir de forma pública (RR.SS., por ejemplo) la información obtenida de esta tarea ya que es el objeto de estudio (y futura publicación) de un ayudante :)

## Un domingo incómodo
En tu último almuerzo familiar, nadie pudo disfrutar mucho la comida ya que antes de esta se generó un momento de tensión cuando tu tio se puso a comentar sobre el borrador de constitución que se está haciendo ahora. Su principal comentario fue "ES IGUAL A LA CONSTITUCIÓN DE VENEZUELA...!!!" Como buen minero de datos, tú le preguntaste de donde obtuvo esa información. "DE TWITTER...!!!" - dijo él. 

Cuando intentaste demostrarle con hechos y lógica que esto no era cierto te percataste de algo: tú tampoco sabías si esto era cierto o no. Por esto, decidiste utilizar todas tus herramientas de Natural Language Processing recientemente adquiridas para analizar este borrador y compararlo con constituciones previamente existentes. 

## 0. Setup

In [ ]:
from IPython.display import clear_output

!pip3 uninstall spacy
!pip3 install spacy

!spacy download es_core_news_lg
clear_output()
print('Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda')

## 1. Importar librerías y descargar dataset
En esta tarea trabajaremos con la librería `spacy` y el pipeline `es_core_news_lg` el cual pesa más de 500 MB y contiene un vocabulario en español de más de medio millón de palabras. Cada una de estas palabras es representable a partir de un vector de 300 dimensiones que nos ayudarán en la tarea. Revisa la [documentación](https://spacy.io/api) documentación de la librería para saber más.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import spacy

nlp = spacy.load("es_core_news_lg")

El siguiente dataframe contiene los artículos de 22 constituciones diferentes de paises americanos y europeos.

In [2]:
url = 'https://drive.google.com/uc?id=1sl8XOW3gez-Gki5v64ddMRNDy-Pwd9qk'
df = pd.read_csv(url, index_col=0)
df

,country,article,text
0,CHL,Artículo 1,Las personas nacen libres e iguales en dignida...
1,CHL,Artículo 2,"Son emblemas nacionales la bandera nacional, e..."
2,CHL,Artículo 3,El Estado de Chile es unitario. La administrac...
3,CHL,Artículo 4,Chile es una república democrática.
4,CHL,Artículo 5,La soberanía reside esencialmente en la Nación...
...,...,...,...
5409,CONV,498 Artículo Nuevo,. - Lo dispuesto en el inciso primero del art...
5410,CONV,499 Artículo 25,. El Estado deberá regular los impactos y efe...
5411,CONV,500 Artículo 30,. Es deber del Estado contribuir y cooperar i...
5412,CONV,501 Artículo nuevo,. El Estado tendrá el deber de fijar una Polí...


## 2. Preprocesamiento (1.5 puntos)
### 2.1 Separar por oraciones
Para cada artículo, tienes que separar en oraciones según los delimitadores `.` y `;`. Se espera un dataframe con 3 columnas (country, article, sentence) con más de 35.000 filas.

In [3]:
new_df = pd.DataFrame(columns=['country', 'article', 'text'])

def split(text):

    splitted = text.split('.')
    splitted = [elements for sentences in splitted for elements in sentences.split(';')]
    
    return splitted
    

df.text = df.text.apply(lambda row: split(row))

for index, row in df.iterrows():
    for sentence in row.text:
        aux_df = pd.DataFrame({'country': [row.country], 'article': [row.article], 'text': [sentence]})
        new_df = pd.concat([new_df, aux_df])
new_df

,country,article,text
0,CHL,Artículo 1,Las personas nacen libres e iguales en dignida...
0,CHL,Artículo 1,La familia es el núcleo fundamental de la soc...
0,CHL,Artículo 1,El Estado reconoce y ampara a los grupos inte...
0,CHL,Artículo 1,El Estado está al servicio de la persona huma...
0,CHL,Artículo 1,Es deber del Estado resguardar la seguridad n...
...,...,...,...
0,CONV,501 Artículo nuevo,establecimiento de la carrera profesional por...
0,CONV,501 Artículo nuevo,y la colaboración entre recintos e infraestru...
0,CONV,501 Artículo nuevo,”\n
0,CONV,502 Artículo 51,


In [4]:
df = new_df.reset_index(drop=True)

### 2.2 Preprocesar oraciones
La función `preprocess` toma una oración, la preprocesa y retorna la oración preprocesada (string). El preproceso incluye 6 subprocesos detallados en el código.

Hint: Al crear un documento con `nlp(sentence)`, cada palabra de la `sentence` tiene atributos que pueden servir al momento de preprocesar.

In [5]:
def remove_punctuation(text):
    # Elimina toda palabra que sea considerada como una puntuación
    text = [token for token in text if not token.is_punct]
    return ' '.join(token.text for token in text)

def remove_stopwords(words):
    # Elimina toda palabra que sea considerada como una stopword a excepción de Estado
    text = [word for word in words.split() if not nlp.vocab[word].is_stop]
    return ' '.join(word for word in text)

def lemmatize(words):
    # Lematiza cada palabra
    doc = nlp(words)
    text = [token.lemma_ for token in doc]
    
    return ' '.join(word for word in text)

def remove_non_alpha(words):
    # Elimina palabras que no sean alfanuméricas
    doc = nlp(words)
    
    text = [word for word in doc if word.is_alpha]
    return ' '.join(word.text for word in text)
    
def lower(words):
    # Lleva toda palabra a minúsculas
    return words.lower()

def min_len(words, length=3):
    # Elimina palabras que tengan un largo menor a length
    text = [word for word in words.split() if len(word) >= length]
    return text

def preprocess(text):

    doc = nlp(text)
    tokens = remove_punctuation(doc)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    tokens = remove_non_alpha(tokens)
    tokens = lower(tokens)
    tokens = min_len(tokens, length=3)
    return ' '.join(tokens).strip()

# Este es un ejemplo para que veas si tu preprocesamiento funcionó.
new_text = preprocess(" i Constitución de Chile: Las personas nacen libres e iguales en dignidad y derechos, La familia es el núcleo fundamental de la sociedad.")
new_text

'constitución chile persona nacer libre igual dignidad derechos familia núcleo fundamental sociedad'

Una correcta función de preprocesamiento retornaría lo siguiente: 

`constitución chile persona nacer libre igual dignidad derecho familia núcleo fundamental sociedad`

Preprocesa toda oración y guarda este vector en una columna del dataframe principal.

In [6]:
df.text = df.text.apply(lambda row: preprocess(row))

### 2.3 Eliminación de datos nulos o inservibles
Elimina datos vacíos y elimina filas con textos de largo pequeño que no son oraciones. Justifica el mínimo de largo seleccionado. 

In [7]:
# Eliminar datos nulos
df = df.dropna()

In [8]:
# Eliminar oraciones de menor largo que tu parámetro seleccionado
df = df[df.text.str.count('\s+') > 5]

In [9]:
# HINT: te recomendamos fuertemente resetear los índices ahora que ya eliminaste varias filas.
df.reset_index(inplace=True, drop=True)

**Justificación:** Creo que dada la gran cantidad de oraciones que poseemos (35 mil aprox) podemos darnos el lujo de dropear aquellas oraciones que no cumplan un con un mínimo de 6 palabras en la oración, por otro lado haber conservado textos cortos conlleva a tener oraciones carentes de contexto, lo cual es perjudicial no solo en machine learning si no incluso en la vida real, donde si hacemos la analogía; un humano podría entender algo totalmente disinto a lo que se buscaba explicar si no lee el contexto y solo se deja llevar por un par de palabras de la oración.

## 3. Vectorizar oraciones (0.5 puntos)
Define una función que dada una oración, retorna un vector que la representa. En esta tarea, el vector de una oración será el promedio de los vectores de cada una de las palabras que fueron preprocesadas de la oración.

In [10]:
def sentence_vector(text):
    doc = nlp(text)
    vs = [word.vector for word in doc]
    vs = np.array(vs)
    
    return vs.mean(axis=0)

Dada la función, crea un nuevo dataframe que contenga los vectores de cada una de las oraciones.

In [11]:
# Tu código aquí

matrix = df.text.apply(lambda row: sentence_vector(row))

In [12]:
n_cols = len(matrix[0])

In [13]:
vectorized_df = pd.DataFrame(np.array([x for x in matrix.values]), columns=[f'c{n}' for n in range(n_cols)])

## 4. Matriz de similitud (1.5 puntos)
### 4.1 Obtención matriz de confusión
Obtén una matriz que por cada par distintos de oraciones contenga la distancia coseno entre los vectores que representan a cada una. Hint: el método `pairwise_distances`  de `sklearn` realiza esta operación eficientemente y no genera problemas de RAM.

In [14]:
from sklearn.metrics import pairwise_distances

similarity_matrix = pairwise_distances(vectorized_df, metric = 'cosine')

### 4.2 Función `print_most_similar`
A partir del nombre de un artículo, imprimir cada una de sus oraciones junto con la oración más parecida que detecta la `similarity_matrix` que sea de otra constitución. Además de la oración, debes imprimir el país de origen de la oración seleccionada.


In [15]:
def print_most_similar(article, X, matrix):
    to_iterate = X[X.article == article]
    
    for index, row in to_iterate.iterrows():
        to_check  = matrix[index][X.country != row.country]
        aux_index = np.where(to_check == np.amin(to_check))
        most_similar = X[X.country != row.country].loc[aux_index]

        print(f'Original text: [{row.country}] {row.text} \n -Most similar [{most_similar.country.values[0]}] {most_similar.text.values[0]}\n\t Cosine similarity {to_check[aux_index]} \n')

In [16]:
print_most_similar('Artículo 1', df, similarity_matrix)

Original text: [CHL] reconocer amparar grupo intermedio organizar estructura sociedad garantizar adecuado autonomía cumplir fin específico 
 -Most similar [MEX] cámaras abrir sesión ejercer cargo concurrencia mitad número miembro
	 Cosine similarity [0.0623191] 

Original text: [CHL] servicio persona humano finalidad promover común contribuir crear condición social permitir integrante comunidad nacional realización espiritual material pleno respeto derechos garantía constitución establecer 
 -Most similar [MEX] fiscalización ingreso egreso partido político candidato determinar ley
	 Cosine similarity [0.04321337] 

Original text: [CHL] deber resguardar seguridad nacional protección población familia propender fortalecimiento promover integración armónico sector nación asegurar derecho persona participar igualdad oportunidad vida nacional 
 -Most similar [URY] enviar mensaje complementario sustitutivo caso exclusivo proyecto presupuesto nacional veinte entrada proyecto cámara
	 Cosine s

### 4.3 Evaluar desempeño
Para cada uno de los 3 artículos del borrador, ejecuta la función y **comenta** sobre los resultados. Se espera un breve análisis por cada uno de los artículos. Todas las oraciones de los artículos del borrador de la convención tienen como `country` a `CONV`.


#### 4.3.1 Sobre la definición del Estado en `428 Artículo 1`

In [17]:
print_most_similar('428 Artículo 1', df, similarity_matrix)

Original text: [CONV] constituir república solidario democracia paritario reconocer valor intrínseco irrenunciable dignidad libertad igualdad sustantivo ser humano relación indisoluble naturaleza 
 -Most similar [VEN] venezuela constituir democrático social derecho justicia propugnar valor superior ordenamiento jurídico actuación vida libertad justicio igualdad solidaridad democracia responsabilidad social general preeminencia derechos humanos ético pluralismo político
	 Cosine similarity [0.08202243] 

Original text: [CONV] protección garantíar derechos humanos individual colectivo fundamento orientar actividad 
 -Most similar [VEN] formular órgano correspondiente recomendación observación necesario protección derechos humanos desarrollar mecanismo comunicación permanente órgano público privado nacional internacional protección defensa derechos humanos
	 Cosine similarity [0.04886037] 

Original text: [CONV] deber generar condición necesario proveer bien servicio asegurar gozar derech

(Comenta aquí)
El articulo habla sobre los derechos humanos y como estos son necesarios e importantes para la sociedad. La similaridad con otros artículos es evidente y pareciera que todos aluden a lo mismo.

#### 4.3.2 Sobre la plurinacionalidad en `130 Artículo 4°`

In [18]:
print_most_similar('130 Artículo 4°', df, similarity_matrix)

Original text: [CONV] chile plurinacional intercultural reconocer coexistencia diverso nación pueblo marco unidad 
 -Most similar [BOL] educación contribuir fortalecimiento unidad identidad plurinacional identidad desarrollo cultural miembro nación pueblo indígén originario campesino entendimiento enriquecimiento intercultural
	 Cosine similarity [0.05722159] 

Original text: [CONV] pueblo nación indígena preexistent mapuche aymara rapa nui lickanantay quechua colla diaguita chango kawashkar yaghan poder reconocido forma establecer ley 
 -Most similar [CHL] declaración candidatura individual caso pueblo mapuche aimara diaguita deber contar patrocinio comunidad asociación indígena registrado corporación nacional desarrollo indígena cacicazgo tradicional reconocido ley
	 Cosine similarity [0.15329045] 



(Comenta aquí) Los artículos hablan sobre reconocer a las distintas comunidades presentes en nuestro pais para promover la plurinacionalidad. En el caso de Bolivia apuntan a lo mismo pero explican como lograrlo (mediante la educación). Mientras que en Chile tambien se aboga a reconocer la independencia de estos pueblos en el marco de la ley pero solo para un grupo reducido (mapuche, aimara y diaguitas).

#### 4.3.3 Sobre el agua en `385 Artículo 26`

In [19]:
print_most_similar('385 Artículo 26', df, similarity_matrix)

Original text: [CONV] constitución garantizar persona derecho agua saneamiento suficiente saludable aceptable asequible accesible 
 -Most similar [ECU] recuperar conservar naturaleza mantener ambiente sano sustentable garantizar persona colectividad acceso equitativo permanente calidad agua aire suelo beneficio recurso subsuelo patrimonio natural
	 Cosine similarity [0.08489287] 

Original text: [CONV] inciso velar satisfacción derecho atender necesidad persona distinto contexto 
 -Most similar [ECU] atender principio interés superior derecho prevalecer persona
	 Cosine similarity [0.06729835] 



(Comenta aquí) El artículo habla sobre garantizar el acceso al agua y atender las necesidades de las personas; ambas comparaciones apuntan al mismo tema de trasnfondo.

## 5. Algoritmo de similaridad entre Constituciones (2.5 puntos)
### 5.1 Implementación `SimilarityCalculator` 
Dada una constitución, se evalúa la similaridad de los artículos con respecto a otras constituciones a partir de la selección de las k oraciones ajenas más parecidas a cada una de las oraciones que constituyen el artículo. A continuación, se presenta el flujo que se debe implementar.

Ejemplo:
Una constitución tiene 10 artículos y definimos k=3.
- Para el artículo 1 (tiene dos oraciones)
    - Para la oración 1
        - Identificamos las k=3 oraciones más parecidad de otras constituciones. Ejemplo: `[CHL, BOL, CHL]`

    - Para la oración 2
      - Identificamos las k=3 oraciones más parecidad de otras constituciones. Ejemplo: `[SWE, CHL, BOL]`

    - Juntamos los resultados de cada oración y quedan las frecuencias: `{CHL : 3, BOL : 2, SWE : 1}`.
    - Como `CHL` es el país que más se repite, concluimos que Chile  es el país que representa al contenido del artículo.

- Realizamos el mismo procedimiento para el resto de los 9 artículos.

- Al contabilizar los países escogidos, podrían quedar las siguientes distribuciones: `{CHL : 80%, SPA : 10%, BOL : 10%}`. Es decir, de los 10 artículos, 8 son catalogados por mayoría como chilenos (80%) y 1 corresponde respectivamente a España y Bolivia (10%)
- Se debe retornar un pandas dataframe que contenga estas últimas frecuencias en porcentaje.




In [20]:
import collections

class SimilarityCalculator:

    def __init__(self, df, matrix):

        self.df = df
        self.matrix = matrix

    def calculate(self, country, k=1):
        df = self.df
        matrix = self.matrix
        # Separar X, y: y contiene las oraciones de country, X al resto
        X = df[df.country != country].reset_index(drop=True)
        y = df[df.country == country]
        
        countries_pct = []
        # Recorre cada artíclo del país
        for name, group in y.groupby('article'):
            k_countries = []
            # Recorre cada oración de un artículo
            for index, row in group.iterrows():
                # Selecciona las k oraciones de X se se parecen más a la oración
                m = matrix[index][df.country != country]
                indices = np.argpartition(m, k)[:k]
                
                k_most_similars = X.loc[indices]
                # Guarda los países de las k oraciones seleccionadas (un país de puede repetir)
                for c in k_most_similars.country.values:
                    k_countries.append(c)
            # El país que más se repita al juntar todas las oraciones será el país del artículo
            countries_pct.append(collections.Counter(k_countries).most_common(1)[0][0])
        # Retorna el porcentaje de que un país en X sea asignado como el más similar a un cierto artículo de country
        counter = collections.Counter(countries_pct)
        cont = sum(counter.values())
        for key in counter.keys():
            counter[key] /= cont
        df = pd.DataFrame(counter.items()).sort_values(by = 1, ascending = False).T
        return df.rename(columns=df.iloc[0]).drop(df.index[0])
        

# Se instancia la clase que implementaste
sim = SimilarityCalculator(df, similarity_matrix)

### 5.2 Resultados
Con un `k=3` muestra la distribución de porcentajes usando a la constitución sueca (`SWE`).

In [21]:
sim.calculate('SWE', k=3)

,MEX,FIN,CHL,COL,ECU,POR,GRE,BRA,GER,CONV,...,BOL,URY,VEN,SPA,PER,NOR,ARG,NED,ITA,USA
1,0.126506,0.120482,0.100402,0.084337,0.076305,0.068273,0.064257,0.056225,0.054217,0.042169,...,0.03012,0.03012,0.024096,0.018072,0.016064,0.016064,0.014056,0.01004,0.006024,0.004016


Realiza lo mismo pero para la constitución venezonala (`VEN`).

In [22]:
sim.calculate('VEN', k=3)

,ECU,CONV,BOL,COL,MEX,CHL,POR,GRE,BRA,SPA,SWE,PER,FRA,FIN,ITA,GER,ARG,URY,NED
1,0.238372,0.133721,0.119186,0.119186,0.110465,0.06686,0.043605,0.031977,0.031977,0.023256,0.020349,0.014535,0.008721,0.008721,0.008721,0.005814,0.005814,0.005814,0.002907


Realiza el procedimiento para el borrador de la constitución `CONV`. Además corre otra instancia que no incluya a la constitución actual `CHL`. Lo anterior es debido a un posible bias que tienen ambas al incluir los strings `Chile, chilenos, chilenas` en sus oraciones.

In [23]:
sim.calculate('CONV', k=3)

,ECU,CHL,BOL,COL,VEN,MEX,POR,BRA,PER,SWE,URY,SPA,GER,FRA,GRE,ARG,FIN,NED,ITA
1,0.257143,0.222449,0.114286,0.093878,0.081633,0.063265,0.059184,0.022449,0.016327,0.014286,0.012245,0.012245,0.008163,0.006122,0.006122,0.004082,0.002041,0.002041,0.002041


In [24]:
# Instancia sin CHL
filtered_similarity_matrix = np.array([x[df.country != 'CHL'] for x in similarity_matrix[df.country != 'CHL']])
filtered_df = df[df.country != 'CHL'].reset_index(drop=True)

sim_sin_CHL = SimilarityCalculator(filtered_df, filtered_similarity_matrix)
sim_sin_CHL.calculate('CONV', k=3)

,ECU,BOL,VEN,COL,MEX,POR,BRA,URY,GRE,GER,SPA,PER,FRA,SWE,FIN,ARG,NED,ITA
1,0.273469,0.138776,0.114286,0.114286,0.102041,0.073469,0.034694,0.02449,0.022449,0.020408,0.018367,0.012245,0.010204,0.010204,0.010204,0.008163,0.006122,0.006122


Ahora, haz el mismo análisis para `CHL`, uno considerando `CONV` y otro sin considerarlo

In [25]:
sim.calculate('CHL', k=3)

,CONV,MEX,COL,POR,VEN,SWE,ECU,URY,GER,BRA,PER,BOL,FRA,FIN,GRE,ARG,ITA
1,0.439189,0.162162,0.081081,0.047297,0.040541,0.040541,0.033784,0.027027,0.02027,0.02027,0.013514,0.013514,0.013514,0.013514,0.013514,0.013514,0.006757


In [26]:
# Instancia sin CONV
filtered_similarity_matrix = np.array([x[df.country != 'CONV'] for x in similarity_matrix[df.country != 'CONV']])
filtered_df = df[df.country != 'CONV'].reset_index(drop=True)

sim_sin_CHL = SimilarityCalculator(filtered_df, filtered_similarity_matrix)
sim_sin_CHL.calculate('CHL', k=3)

,MEX,COL,URY,VEN,ECU,BOL,SWE,POR,GRE,FIN,BRA,SPA,PER,ARG,GER,NED
1,0.283784,0.216216,0.081081,0.067568,0.047297,0.047297,0.047297,0.047297,0.033784,0.027027,0.02027,0.02027,0.02027,0.02027,0.013514,0.006757


### 5.3 Tweets
Con tu misión de análisis completada, decides intentar publicar información verdadera en twitter, para que tu tío pueda leer algo que no sea sólo propaganda política engañosa.

A partir de los resultados obtenidos, redacta tres tweets de hasta 140 **caracteres** (no palabras) que agreguen valor a la discusión pública de cara al plebiscito de salida. No debes publicarlos en Twitter, sino que simplemente escribirlos acá. Independiente de tu posición política, esperamos un comentario respetuoso y basado en datos. Puedes expandir tus comentarios a otros países no evaluados en la sección anterior.


### 5.4 Posibles sesgos y mejoras
Comenta también sobre posibles sesgos en el preprocesamiento de los datos, la vectorización de las oraciones y del algoritmo de similaridad que implementaste. Para cada uno de los tres casos, propón al menos una mejora.

## 6. Bonus (hasta un punto extra)
Juega con tu creatividad y diseña una visualización interesante que analice datos obtenidos en la tarea. Se espera tanto creatividad como calidad gráfica de la visualización.